In [1]:
import os
import openai
import sys
import langchain
from langchain_openai import OpenAI

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.environ['OPENAI_API_KEY']

langchain.debug=True

In [15]:
# LANGCHAIN_API_KEY=os.environ['LANGCHAIN_API_KEY']
# os.environ["LANGCHAIN_TRACING"] = "true"

# %env LANGCHAIN_TRACING_V2=true
# %env LANGCHAIN_ENDPOINT="https://api.smith.langchain.com"
# %env LANGCHAIN_API_KEY="ls__cf8ee1e5c338467eb25668ff51f4548c"
# %env LANGCHAIN_PROJECT="YTU_Chatbot"

# Retrieval

### Loading Vector database

In [2]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma

embedding = OpenAIEmbeddings()

ytu_vectordb = Chroma(persist_directory='./db/ytu_website', embedding_function=embedding)
end_vectordb = Chroma(persist_directory='./db/end_website', embedding_function=embedding)
erasmus_vectordb = Chroma(persist_directory='./db/erasmus_website', embedding_function=embedding)

### Simple Retrieval

In [3]:
Question = 'Üniversitenin Misyonu ve Vizyonu nedir '
answers = ytu_vectordb.similarity_search(Question, k=2)

for ans in answers:
    print(ans.page_content)
    print('-------------------------------------------------------------------')

Source: https://www.yildiz.edu.tr/universite/universitemiz/misyon-ve-vizyon
Page Title: Misyon ve Vizyon | Yıldız Teknik Üniversitesi
Subject: universitemiz / misyon-ve-vizyon

Misyon ve Vizyon | Yıldız Teknik ÜniversitesiÜniversiteÜniversitemizMisyon ve VizyonMisyon ve Vizyon
A+
A-

  
Misyon
Uluslararası düzeyde eğitim, öğretim ve araştırma mükemmelliğine ulaşarak toplumun ihtiyaçlarına ve ulusal rekabet gücüne katkıda bulunan bireyler yetiştirmek, yenilikçi fikirleri ve uygulamaları hayata geçirmektir.
 
Vizyon
Çözüm üretme, yön verme ve dönüştürme sorumluluğumuzu arttırarak bölgesel ve küresel sürdürülebilir kalkınmaya olan belirgin katkımızı güçlendiriyoruz.
  Geri
            
Paylaş   

ÜNİVERSİTE
ÜniversitemizTarihçeMisyon ve VizyonKampüslerAkademik KadroKurumsal KimlikSayılarla YTÜYTÜ RektörleriYönetimRektörRektör YardımcılarıYönetim KuruluSenatoGenel SekreterlikHukuk Müşavirliğiİç Denetim BirimiInternational Advisory Board

Daire BaşkanlıklarıKoordinatörlükler
---------------

In [5]:
Question = 'lisans Mesleki Alan Stajı 1'
answers = end_vectordb.similarity_search(Question, k=4)

for ans in answers:
    print(ans.page_content)
    print('-------------------------------------------------------------------')

Source: https://enm.yildiz.edu.tr/lisans/mesleki-alan-staji-1
Page Title: Mesleki Alan Stajı 1 | YTÜ Endüstri Mühendisliği Bölümü
Subject: lisans / mesleki-alan-staji-1

Mesleki Alan 1 Stajı için başvuru ve teslim sürecinde yapılması gerekenler ve süreçte lazım olacak belgeler aşağıdaki akış şemasında verilmiştir. 
 
MAS-1 Başvuru ve Teslim Süreci Akış Şeması
 
Mesleki Alan 1 (Bilgi Teknolojileri) Stajı hakkında genel bilgileri ve sıkça sorulan soruları içeren Staj Uygulama İlkeleri için aşağıdaki bağlantıları inceleyiniz. Mesleki Alan 1 Stajı için tanımlı işletme listesi bulunmamaktadır. İşletme tanıtımı için, Mesleki Alan 1 stajı için Tanıtım Formu doldurunuz. Formu gönderdikten sonra e-posta adresinize, adınıza düzenlenmiş Mesleki Alan 1 Staj Yeri Tanıtım Formu gönderilir. Bu formun çıktısı alınarak şirket tarafından onaylanması (imzalı+kaşeli) gerekmektedir. Onaylanan Staj Yeri Tanıtım Formu’nu Mesleki Alan 1 Stajı Değerlendirme Komisyonu’nun (endstajma1@gmail.com) onayına sunmalıd

In [3]:
Question = 'Erasmus+ program başvuruları ne zaman başlıyor?'
answers = erasmus_vectordb.similarity_search(Question, k=3)

for ans in answers:
    print(ans.page_content)
    print('-------------------------------------------------------------------')

Source: https://erasmus.yildiz.edu.tr/sayfa/7/2024-2025-Başvuruları-için-Süreç-Takvimi/911
Page Title:  Erasmus + Program Birimi Resmi Sitesi
Subject: 2024-2025-Başvuruları-için-Süreç-Takvimi

. Yine Staj hareketliliğinde ek puan almak için de başvuru tarihlerinde staj kabul belgesi yüklemeniz gerekmektedir. Ayrıntılar başvuru çağrı metninde yer almaktadır. Usulüne uygun yüklenmeyen belgeler geçersiz sayılacaktır.Ön başvuru yapan öğrencilerin ilanı5 Ocak 2024Erasmus+ Program Birimi web sayfasında ön başvuru yapan tüm adayların bilgilerinin bulunduğu Ön Başvuru Listesi yayınlanacaktır.Çift başvuru puan kesintisi dilekçelerinin Erasmus Ofisine teslimi08-14 Ocak 2024 Her iki faaliyet türüne (Öğrenim ve Staj) birlikte başvuran öğrencilerden, hangi faaliyet türü başvurusundan puan kesileceği konusunda dilekçe alınacaktır. Dilekçe online ortamda doldurup, imzalanıp PDF olarak başvuru yapılan YTÜ Erasmus Otomasyon sistemine ytuerasmus.yildiz.edu.tr Çift Başvuru Dilekçesi kısmına yüklenmelidir

# LangChain

<img src='https://miro.medium.com/v2/resize:fit:1200/1*A2M2OqVP-s6aA9IqDcgaRQ.png'>

## Routing

In [4]:
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor 
from langchain.retrievers.document_compressors import EmbeddingsFilter
from langchain.retrievers.document_compressors import LLMChainFilter
from langchain.retrievers.document_compressors import DocumentCompressorPipeline


# Filters and Compressers
llm = OpenAI(temperature=0)

relevant_filter = LLMChainFilter.from_llm(llm)
embeddings_filter = EmbeddingsFilter(embeddings=OpenAIEmbeddings(), similarity_threshold=0.85)
compressor = LLMChainExtractor.from_llm(llm)
pipeline_compressor = DocumentCompressorPipeline(
    transformers=[embeddings_filter, relevant_filter]
)

base_compressor = pipeline_compressor


# Build prompt
erasmus_template = PromptTemplate.from_template(
"""You are Yildiz Technical University' helpful chatbot, called 'YTU Chatbot'.\
You have a funny and energetic spirit. The students, staff from this university \
will ask you specifically about Erasmus Program. Respond to them joyfully and helpfully.
For every question asked, you will be provided with a context to assist you in answering.\
The context has been retrieved from the scraped University Erasmus Page.
Questions and context might be in Turkish.

Use the following pieces of context to answer the question at the end. \
If you don't know the answer, just say that you don't know,\
don't try to make up an answer. Give a detailed answer.
{context}
Question: {question}
Helpful Answer:""")

end_template = PromptTemplate.from_template(
"""You are Yildiz Technical University' helpful chatbot, called 'YTU Chatbot'.\
You have a funny and energetic spirit. The students, staff from this university \
will ask you specifically about Industrial Engineering major. Respond to them joyfully\
and helpfully. For every question asked, you will be provided with a context to\
assist you in answering. The context has been retrieved from the\
scraped University Industrial Engineering Page.
Questions and context might be in Turkish.

Use the following pieces of context to answer the question at the end. \
If you don't know the answer, just say that you don't know,\
don't try to make up an answer. 
{context}
Question: {question}
Helpful Answer:""")

ytu_template = PromptTemplate.from_template(
"""You are Yildiz Technical University' helpful chatbot, called 'YTU Chatbot'.\
You have a funny and energetic spirit. The students, staff from this university \
will ask you various questions. Respond to them joyfully and funnyly.
For every question asked, you will be provided with a context to assist you in answering.\
The context has been retrieved from the scraped university's general page.
Questions and contexts might be in Turkish.
REMEMMBER! Always greet them at the beggining of the Helpful Answer.

Use the following pieces of context to answer the question at the end. \
If you don't know the answer, just say that you don't know,\
don't try to make up an answer. Give a detailed answer. 
{context}
Question: {question}
Helpful Answer:""")



end_compression_retriever = ContextualCompressionRetriever(
    base_compressor=base_compressor,
    base_retriever=end_vectordb.as_retriever(k=1)
)
ytu_compression_retriever = ContextualCompressionRetriever(
    base_compressor=base_compressor,
    base_retriever=ytu_vectordb.as_retriever()
)
erasmus_compression_retriever = ContextualCompressionRetriever(
    base_compressor=base_compressor,
    base_retriever=erasmus_vectordb.as_retriever()
)

# END chain
end_qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=end_compression_retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt": end_template}
)
# Erasmus chain
erasmus_qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=erasmus_vectordb.as_retriever(),
    return_source_documents=True,
    chain_type_kwargs={"prompt": erasmus_template}
)

# YTU chain
ytu_qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=ytu_compression_retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt": ytu_template}
)

In [5]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableLambda

gpt_llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)
chain = (
    PromptTemplate.from_template(
        """You are yildiz technical university chatbot. \
        You will be given a student question below, \
        classify it as either being about `General`, `Major`, or `Erasmus`.
        The `Major` classification will be if the student asks questions \
        related to their major. The `Erasmus` classification if they only \
        asked about Erasmus. The `General` classification will be if they asked\
        generally about university and stuff.

Do not respond with more than one word.

<question>
{question}
</question>

Classification:"""
    )
    | gpt_llm
    | StrOutputParser()
)

#-----------------------------------------------------



#-----------------------------------------------------

def route(info):
    if "erasmus" in info["topic"].lower():
        return erasmus_qa_chain
    elif "major" in info["topic"].lower():
        return end_qa_chain
    else:
        return ytu_qa_chain

#-----------------------------------------------------


full_chain = {"topic": chain, "question": lambda x: x["question"]} | RunnableLambda(route)


In [23]:
query1 = 'Lisans Mesleki alan staj 1 hangi konu ile ilgili olacak ? ne içeriyor ?'
query2 = 'Universitede olan faydalanacağım programlar nedir ?'
query3 = 'KOOP programı ne ?'
query4 = 'Erasmus Programın başvuru tarihleri nelerdir ?'
query5 = 'Merhaba'
query6 = 'Üniversitede bana tavsiye edebileceğin önemli ve enteresan faayliyet ve etkinlikler var mı ?'

query10 = 'Üniversitenin Misyonu ve Vizyonu nedir ?'
query100 = 'lisans Mesleki Alan Stajı 1 ne ile ilgil ?'
query1000 = 'Erasmus+ staj 1. Periyod için kabul mektubun son teslim tarihi nedir?'

answer = full_chain.invoke({"question": query1000})
print(answer['result'])

[chain/start] [1:chain:RunnableSequence] Entering Chain run with input:
{
  "question": "Erasmus+ staj 1. Periyod için kabul mektubun son teslim tarihi nedir? (1. periyod kabul mektubu son teslimi 2. periyod değil. Bilmiyorsan ya da bulamadıysan de)"
}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableParallel<topic,question>] Entering Chain run with input:
{
  "question": "Erasmus+ staj 1. Periyod için kabul mektubun son teslim tarihi nedir? (1. periyod kabul mektubu son teslimi 2. periyod değil. Bilmiyorsan ya da bulamadıysan de)"
}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableParallel<topic,question> > 3:chain:RunnableSequence] Entering Chain run with input:
{
  "question": "Erasmus+ staj 1. Periyod için kabul mektubun son teslim tarihi nedir? (1. periyod kabul mektubu son teslimi 2. periyod değil. Bilmiyorsan ya da bulamadıysan de)"
}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableParallel<topic,question> > 3:chain:RunnableSequence > 4:prompt:PromptT

[llm/end] [1:chain:RunnableSequence > 7:chain:route > 8:chain:RetrievalQA > 10:chain:StuffDocumentsChain > 11:chain:LLMChain > 12:llm:OpenAI] [3.11s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": " 1. Periyod için kabul mektubunun son teslim tarihi 27 Ocak 2024 saat 16:30'dur. Bu tarihten sonra gönderilen belgeler kabul edilmeyecektir. Lütfen bu tarihe dikkat ederek kabul mektubunuzu zamanında yükleyin. Ayrıca, kabul mektubunuzun asgari içerik ve imza şartlarını sağlaması gerektiğini unutmayın. Daha fazla bilgi için web sitemizi ziyaret edebilirsiniz.",
        "generation_info": {
          "finish_reason": "stop",
          "logprobs": null
        },
        "type": "Generation"
      }
    ]
  ],
  "llm_output": {
    "token_usage": {
      "completion_tokens": 143,
      "total_tokens": 2104,
      "prompt_tokens": 1961
    },
    "model_name": "gpt-3.5-turbo-instruct"
  },
  "run": null
}
[chain/end] [1:chain:RunnableSequence > 7:chain:route > 8:

In [24]:
answer['result']

" 1. Periyod için kabul mektubunun son teslim tarihi 27 Ocak 2024 saat 16:30'dur. Bu tarihten sonra gönderilen belgeler kabul edilmeyecektir. Lütfen bu tarihe dikkat ederek kabul mektubunuzu zamanında yükleyin. Ayrıca, kabul mektubunuzun asgari içerik ve imza şartlarını sağlaması gerektiğini unutmayın. Daha fazla bilgi için web sitemizi ziyaret edebilirsiniz."

In [25]:
for doc in answer['source_documents']:
    print(doc.page_content)
    print('\n\n')

Source: https://erasmus.yildiz.edu.tr/haberler/625/2023-2024-Akademik-Yılı-KA131-Staj-2--Periyod-Kabul-Belgesi-Teslimi-ve-Dezavantajlı-Geçmiş-Belgesi-Hakkında-11-01-2024
Page Title:  Erasmus + Program Birimi Resmi Sitesi
Subject: 2023-2024-Akademik-Yılı-KA131-Staj-2--Periyod-Kabul-Belgesi-Teslimi-ve-Dezavantajlı-Geçmiş-Belgesi-Hakkında-11-01-2024

Kabul Belgesi (Erasmus+ Program Birimine teslim süreci).2. Periyod kabul belgesi teslim tarihleri 23 – 27 Ocak 2024 arasıdır. 27 Ocak 2024 saat 16:30'a kadar tüm sürecin tamamlanması gerekmektedir.Bu tarihler dışında gönderilen belgeler kabul edilmeyecektir.Asgari içerik ve imza şartlarını sağlamayan belgeler kabul edilmeyecektir Detaylı bilgiye buradan ulaşabilirsiniz.Aldığınız davet yazısını Bölüm Erasmus Koordinatörünüze onaylatmayı unutmayınız.Kabul belgenizi ytuerasmus.yildiz.edu.tr  adresine Kabul Mektubu kısmına yüklemeniz yeterlidir. E-posta göndermenize gerek bulunmamaktadır.Başvuru esnasında Kabul Mektubu yükleyen öğrencilerin yenid